In [1]:
import torch
import numpy as np
import eznf
from eznf import Tensor
from eznf import datasets
from eznf.autograd import function
# from eznf.nn.modules import *

In [5]:
class Linear(eznf.nn.Module):
    def __init__(self, in_features, out_features):
        super().__init__()
        self.w = Tensor(in_features, out_features, requires_grad=True, is_leaf=True)

    def forward(self, x):
        return self.w @ x

In [7]:
Linear(5, 20).w

tensor(
[[ 1.36716722 -0.91854857 -0.29956372  0.32508596 -0.03724147 -0.32652584
   0.82737627 -0.56760402 -1.6011445   1.33811853  1.0079868  -1.11937218
  -0.27191409 -0.21947478  0.10788551  0.87125386 -0.25388602  0.75616345
  -1.66674723 -0.80987213]
 [ 2.37099842  0.84208844 -0.47129915  1.83829825  1.28006438 -1.55810657
   1.1699201   1.32384169  1.08853656 -1.47434713 -2.29149502  0.35764747
   0.50111118 -0.41071073 -0.1421328   0.30215097 -1.12809527 -0.48940214
   0.42080944 -0.49662335]
 [-1.97654574  0.28698248  0.16927401  0.72724936  0.22167246 -1.1661354
  -1.27824746  0.2384094   0.24272678 -0.36758297  0.24779637 -2.34836121
   0.79643592 -0.20815625  0.14333092 -0.36379853 -0.31689589 -1.51781777
  -0.88924273  0.26832825]
 [ 1.33316787 -0.40050651 -0.09283801  0.05972702  0.67316232  0.35953236
  -1.87661789  1.08968832 -0.21334687  0.38923724  1.3486157   0.68901541
  -0.02166691 -1.27055485  0.90038633 -0.56040382  0.79901462 -1.59363429
  -0.91111399 -1.1946688

In [ ]:
torch.Tensor(device='cuda')
import cupy as np